In [0]:
from pyspark.sql.functions import col,isnan, when, count

In [0]:
my_df = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/vivek.kaushik.bit@gmail.com/ol_cdump.json")

In [0]:
my_df.count()

Out[3]: 148163

In [0]:
#display(my_df)

In [0]:
#Adding filters:

In [0]:
from datetime import date  
# creating the date object of today's date
todays_date = date.today()  
# fetching the current year, month and day of today
print("Current year:", todays_date.year)
this_year=todays_date.year

Current year: 2022

In [0]:
my_clean_df=my_df.filter(col('title').isNotNull()).filter(col('number_of_pages')>20).filter(col('publish_date')>1950)

In [0]:
my_clean_df.count()

Out[7]: 75439

In [0]:
my_clean_df=my_df.filter(col('title').isNotNull()).filter(col('number_of_pages')>20).filter((col('publish_date')>1950) & (col('publish_date')<=this_year))

In [0]:
my_clean_df.count()

Out[9]: 75429

In [0]:
my_df.filter(col('title').isNotNull()).filter(col('number_of_pages')>20).filter((col('publish_date')>2022)).select('title','number_of_pages','publish_date').show()

+--------------------+---------------+------------+
 title|number_of_pages|publish_date|
+--------------------+---------------+------------+
A century of Jewi...| 814| 5704|
Bodies are only t...| 65| 2972|
Alternating currents| 51| 9999|
The quest for the...| 128| 9999|
About some holida...| 31| 9999|
Jewish pioneers a...| 430| 5703|
 Symphony, no. 3| 168| 3606|
 Lieder ohne Worte.| 145| 8727|
Overture to the r...| 34| 3707|
 Anthology.| 260| 9971|
+--------------------+---------------+------------+

In [0]:
#Get the book with the most pages:

In [0]:
row1 = my_clean_df.agg({"number_of_pages": "max"}).collect()[0]
max_num_pages=row1["max(number_of_pages)"]
max_num_pages_df= my_clean_df.filter(col('number_of_pages')==max_num_pages)

In [0]:
display(max_num_pages_df.select('title','number_of_pages'))

title,number_of_pages
Nihon shokuminchi kenchikuron,48418


In [0]:
display(max_num_pages_df)

alternate_names,authors,bio,birth_date,by_statement,contributions,contributors,copyright_date,covers,created,death_date,description,dewey_decimal_class,dewey_number,download_url,edition_name,excerpts,first_publish_date,first_sentence,full_title,fuller_name,genres,ia_box_id,ia_loaded_id,identifiers,isbn_10,isbn_13,isbn_invalid,isbn_odd_length,key,languages,last_modified,latest_revision,lc_classifications,lccn,links,location,name,notes,number_of_pages,ocaid,oclc_number,oclc_numbers,other_titles,pagination,personal_name,photos,physical_dimensions,physical_format,publish_country,publish_date,publish_places,publishers,purchase_url,revision,series,source_records,subject_people,subject_place,subject_places,subject_time,subject_times,subjects,subtitle,table_of_contents,title,title_prefix,type,uri_descriptions,uris,url,website,weight,work_title,work_titles,works
null,"List(List(null, /authors/OL5510271A, null))",null,null,Nishizawa Yasuhiko cho.,null,null,null,null,"List(/type/datetime, 2009-01-07T22:47:41.819566)",null,null,null,null,null,880-03 Shohan.,null,null,null,null,null,null,null,null,null,List(4815805806),List(9784815805807),null,null,/books/OL22855337M,List(List(/languages/jpn)),"List(/type/datetime, 2009-12-15T08:07:24.201731)",2,List(NA1555 .N575 2008),List(2008543302),null,null,null,"{""type"":""/type/text"",""value"":""Includes bibliographical references and index.""}",48418,null,List(216939675),null,null,"x, 484,18 p. :",null,null,null,null,ja,2008,List(Nagoya-shi),List(Nagoya Daigaku Shuppankai),null,2,null,null,null,null,null,null,null,"List(Architecture -- Japan -- Colonies -- History., Architecture -- East Asia -- History.)",null,null,Nihon shokuminchi kenchikuron,null,List(/type/edition),null,null,null,null,null,null,null,List(List(/works/OL12468635W))


In [0]:
#method2

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *


w = Window.orderBy(desc('number_of_pages'))
my_clean_df_interim=my_clean_df.withColumn('Rank',dense_rank().over(w))
my_clean_df_interim=my_clean_df_interim.filter(my_clean_df_interim.Rank == 1).drop(my_clean_df_interim.Rank)

In [0]:
display(my_clean_df_interim.select('title','number_of_pages'))

title,number_of_pages
Nihon shokuminchi kenchikuron,48418


In [0]:
display(my_clean_df_interim)

alternate_names,authors,bio,birth_date,by_statement,contributions,contributors,copyright_date,covers,created,death_date,description,dewey_decimal_class,dewey_number,download_url,edition_name,excerpts,first_publish_date,first_sentence,full_title,fuller_name,genres,ia_box_id,ia_loaded_id,identifiers,isbn_10,isbn_13,isbn_invalid,isbn_odd_length,key,languages,last_modified,latest_revision,lc_classifications,lccn,links,location,name,notes,number_of_pages,ocaid,oclc_number,oclc_numbers,other_titles,pagination,personal_name,photos,physical_dimensions,physical_format,publish_country,publish_date,publish_places,publishers,purchase_url,revision,series,source_records,subject_people,subject_place,subject_places,subject_time,subject_times,subjects,subtitle,table_of_contents,title,title_prefix,type,uri_descriptions,uris,url,website,weight,work_title,work_titles,works
null,"List(List(null, /authors/OL5510271A, null))",null,null,Nishizawa Yasuhiko cho.,null,null,null,null,"List(/type/datetime, 2009-01-07T22:47:41.819566)",null,null,null,null,null,880-03 Shohan.,null,null,null,null,null,null,null,null,null,List(4815805806),List(9784815805807),null,null,/books/OL22855337M,List(List(/languages/jpn)),"List(/type/datetime, 2009-12-15T08:07:24.201731)",2,List(NA1555 .N575 2008),List(2008543302),null,null,null,"{""type"":""/type/text"",""value"":""Includes bibliographical references and index.""}",48418,null,List(216939675),null,null,"x, 484,18 p. :",null,null,null,null,ja,2008,List(Nagoya-shi),List(Nagoya Daigaku Shuppankai),null,2,null,null,null,null,null,null,null,"List(Architecture -- Japan -- Colonies -- History., Architecture -- East Asia -- History.)",null,null,Nihon shokuminchi kenchikuron,null,List(/type/edition),null,null,null,null,null,null,null,List(List(/works/OL12468635W))


In [0]:
#Find the top 5 genres with most books:

In [0]:
my_clean_df_genre=my_clean_df.select('title',explode('genres').alias('Genres'))

In [0]:
#display(my_clean_df_genre)

In [0]:
my_clean_df_genre=my_clean_df_genre.groupBy('Genres').count()
display(my_clean_df_genre)

Genres,count
Art collections,1
Personal narratives,16
Controversial literature,2
music.,1
Folklore,27
Congressess.,1
Library,1
Fiction.,3300
Interviews.,133
Early works to 1800,17


In [0]:
w = Window.orderBy(desc('count'))
my_clean_df_genre_interim=my_clean_df_genre.withColumn('Rank',dense_rank().over(w))
my_clean_df_genre_interim=my_clean_df_genre_interim.filter(my_clean_df_genre_interim.Rank <= 5).drop(my_clean_df_genre_interim.Rank)

In [0]:
display(my_clean_df_genre_interim)

Genres,count
Fiction.,3300
Biography.,2360
Juvenile literature.,1538
Exhibitions.,836
Juvenile fiction.,525


In [0]:
#Retrieve the top 5 authors who (co-)authored the most books:

In [0]:
my_clean_df_coauth=my_clean_df.select('title',explode('authors'))

In [0]:
#display(my_clean_df_coauth)

In [0]:
my_clean_df_coauth.printSchema()

root
-- title: string (nullable = true)
-- col: struct (nullable = true)
 |-- author: struct (nullable = true)
 | |-- key: string (nullable = true)
 |-- key: string (nullable = true)
 |-- type: string (nullable = true)

In [0]:
my_clean_df_coauth=my_clean_df_coauth.select(col('title'),col('col.author.key').alias('author_key'),col('col.key'),col('col.type'))

In [0]:
my_clean_df_coauth.printSchema()

root
-- title: string (nullable = true)
-- author_key: string (nullable = true)
-- key: string (nullable = true)
-- type: string (nullable = true)

In [0]:
#display(my_clean_df_coauth)

In [0]:
my_clean_df_coauth.filter(col('author_key').isNotNull()).count()

Out[29]: 0

In [0]:
display(my_df.filter(col('title')=='Cancionero de Puerto Rico'))

alternate_names,authors,bio,birth_date,by_statement,contributions,contributors,copyright_date,covers,created,death_date,description,dewey_decimal_class,dewey_number,download_url,edition_name,excerpts,first_publish_date,first_sentence,full_title,fuller_name,genres,ia_box_id,ia_loaded_id,identifiers,isbn_10,isbn_13,isbn_invalid,isbn_odd_length,key,languages,last_modified,latest_revision,lc_classifications,lccn,links,location,name,notes,number_of_pages,ocaid,oclc_number,oclc_numbers,other_titles,pagination,personal_name,photos,physical_dimensions,physical_format,publish_country,publish_date,publish_places,publishers,purchase_url,revision,series,source_records,subject_people,subject_place,subject_places,subject_time,subject_times,subjects,subtitle,table_of_contents,title,title_prefix,type,uri_descriptions,uris,url,website,weight,work_title,work_titles,works
null,"List(List(List(/authors/OL5360212A), null, {""key"":""/type/author_role""}))",null,null,null,null,null,null,null,"List(/type/datetime, 2009-12-11T06:30:00.004446)",null,null,null,null,null,null,null,1954,null,null,null,null,null,null,null,null,null,null,null,/works/OL12268504W,null,"List(/type/datetime, 2010-12-03T19:10:20.603637)",3,List(M1621.F47 C35 1954),null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,null,null,null,null,List(Puerto Rico),null,null,"List(Folk songs, Spanish, Songs (High voice) with piano, Spanish Folk songs)",null,null,Cancionero de Puerto Rico,null,List(/type/work),null,null,null,null,null,null,null,null


In [0]:
my_clean_df_coauth_grped=my_clean_df_coauth.groupBy('key').count()

In [0]:
#display(my_clean_df_coauth_grped.orderBy(col('count').desc()))

In [0]:
my_clean_df_coauth_grped.orderBy(col('count').desc()).show()

+-------------------+-----+
 key|count|
+-------------------+-----+
/authors/OL1224818A| 236|
/authors/OL4283462A| 116|
 /authors/OL785848A| 106|
/authors/OL1926829A| 80|
 /authors/OL883775A| 75|
 /authors/OL171521A| 72|
 /authors/OL432880A| 56|
/authors/OL4287637A| 54|
 /authors/OL113611A| 53|
 /authors/OL724353A| 49|
/authors/OL6559140A| 48|
/authors/OL4900362A| 47|
/authors/OL6532917A| 46|
 /authors/OL161167A| 45|
/authors/OL1162080A| 42|
/authors/OL4282449A| 38|
/authors/OL5004509A| 37|
/authors/OL5268670A| 37|
 /authors/OL546900A| 37|
/authors/OL4361397A| 35|
+-------------------+-----+
only showing top 20 rows

In [0]:
w = Window.orderBy(desc('count'))
my_clean_df_coauth_grped_interim=my_clean_df_coauth_grped.withColumn('Rank',dense_rank().over(w))
my_clean_df_coauth_grped_interim=my_clean_df_coauth_grped_interim.filter(my_clean_df_coauth_grped_interim.Rank <= 5).drop(my_clean_df_coauth_grped_interim.Rank)

In [0]:
display(my_clean_df_coauth_grped_interim)

key,count
/authors/OL1224818A,236
/authors/OL4283462A,116
/authors/OL785848A,106
/authors/OL1926829A,80
/authors/OL883775A,75


In [0]:
#Per publish year, get the number of authors that published at least one book:

In [0]:
my_clean_df_publish=my_clean_df.select('title','publish_date',explode('authors'))

In [0]:
#df = df.filter(df.colD.between(200, 400))

In [0]:
#display(my_clean_df_publish)

In [0]:
my_clean_df_publish.printSchema()

root
-- title: string (nullable = true)
-- publish_date: string (nullable = true)
-- col: struct (nullable = true)
 |-- author: struct (nullable = true)
 | |-- key: string (nullable = true)
 |-- key: string (nullable = true)
 |-- type: string (nullable = true)

In [0]:
my_clean_df_publish_flat=my_clean_df_publish.select(col('title'),'publish_date',col('col.author.key').alias('author_key'),col('col.key'),col('col.type'))

In [0]:
my_clean_df_publish_flat.printSchema()

root
-- title: string (nullable = true)
-- publish_date: string (nullable = true)
-- author_key: string (nullable = true)
-- key: string (nullable = true)
-- type: string (nullable = true)

In [0]:
#display(my_clean_df_publish_flat)

In [0]:
my_clean_df_publish_flat.filter(col('key').isNull()).count()

Out[43]: 0

In [0]:
my_clean_df_publish_flat.filter(col('author_key').isNotNull()).count()

Out[44]: 0

In [0]:
my_clean_df_publish_flat.count()

Out[45]: 75349

In [0]:
my_clean_df_publish_flat.groupBy('publish_date').count().show()

+------------+-----+
publish_date|count|
+------------+-----+
 1953| 622|
 1957| 740|
 1987| 1279|
 1956| 664|
 1958| 789|
 1972| 1081|
 1988| 1300|
 1977| 755|
 1971| 1136|
 1984| 1106|
 1982| 970|
 2005| 1217|
 2000| 1071|
 1965| 1347|
 1962| 1128|
 1954| 666|
 1981| 871|
 1978| 789|
 1974| 1200|
 1959| 879|
+------------+-----+
only showing top 20 rows

In [0]:
#df = df.filter(df.colD.between(200, 400))
#my_clean_df_publish_flat = my_clean_df_publish_flat.filter(my_clean_df_publish_flat.publish_date.between(1950, 1970))

In [0]:
#display(my_clean_df_publish_flat)

In [0]:
#display(my_clean_df_publish_flat.groupBy('publish_date','key').count())

In [0]:
my_clean_df_publish_flat_grp=my_clean_df_publish_flat.groupBy('publish_date','key').count()

In [0]:
#display(my_clean_df_publish_flat_grp)

In [0]:
display(my_clean_df_publish_flat_grp.groupBy('publish_date').count().orderBy('publish_date'))

publish_date,count
1951,648
1952,601
1953,600
1954,644
1955,588
1956,638
1957,705
1958,753
1959,830
1960,920


In [0]:
display(my_clean_df_publish_flat_grp.groupBy('publish_date').count())

publish_date,count
1953,600
1957,705
1987,1153
1956,638
1958,753
1972,1010
1988,1212
1977,704
1971,1070
1984,1010


In [0]:
#Find the number of authors and number of books published per month for years between 1950 and 1970:

In [0]:
my_clean_df_publish_pmonth=my_clean_df.select('title','publish_date','first_publish_date',explode('authors'))

In [0]:
#display(my_clean_df_publish_pmonth)
my_clean_df_publish_pmonth.show(5)

+--------------------+------------+------------------+--------------------+
 title|publish_date|first_publish_date| col|
+--------------------+------------+------------------+--------------------+
The effect of dif...| 1975| null|[, /authors/OL653...|
Comparison of the...| 1976| null|[, /authors/OL653...|
Professional acci...| 1977| null|[, /authors/OL653...|
I chauceriani sco...| 1964| null|[, /authors/OL590...|
Lezioni zurighesi...| 1955| null|[, /authors/OL135...|
+--------------------+------------+------------------+--------------------+
only showing top 5 rows

In [0]:
my_clean_df_publish_pmonth.filter(col('first_publish_date').isNotNull()).count()

Out[56]: 0

In [0]:
my_df.filter(col('created').isNotNull()).count()

Out[57]: 146000

In [0]:
#display(my_df.filter(col('created').isNotNull()).select('created').orderBy('created'))
my_df.filter(col('created').isNotNull()).select('created').orderBy('created').show(5,truncate=False)

+--------------------------------------------+
created |
+--------------------------------------------+
[/type/datetime, 2008-04-01T03:28:50.625462]|
[/type/datetime, 2008-04-01T03:28:50.625462]|
[/type/datetime, 2008-04-01T03:28:50.625462]|
[/type/datetime, 2008-04-01T03:28:50.625462]|
[/type/datetime, 2008-04-01T03:28:50.625462]|
+--------------------------------------------+
only showing top 5 rows

In [0]:
#display(my_df.filter(col('last_modified').isNotNull()))

In [0]:
#Unable to understand last problem since did not find any relevant column containing publish date in dd/mm/yyyy format. All publish date delated are only years 